In [ ]:
from skimage.segmentation import mark_boundaries, slic
from skimage.util import img_as_float
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import argparse
import os

In [ ]:
def cambio_tam(img, mask_m, mask_h, mask_e):

    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    gray = cv.GaussianBlur(gray, (7, 7), 3)
    t, dst = cv.threshold(gray, 0, 255, cv.THRESH_BINARY | cv.THRESH_TRIANGLE)
    contours, _= cv.findContours(dst, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    a = []

    for c in contours:
        area = cv.contourArea(c)
        a.append(area)
        
    for f in contours:
        area = cv.contourArea(f)
        if area == max(a):
            x, y, w, h = cv.boundingRect(f)
            imgr = img[y:y + h, x:x + w]
            mask_f1= mask_m[y:y + h, x:x + w]
            mask_f2= mask_h[y:y + h, x:x + w]
            mask_f3= mask_e[y:y + h, x:x + w]

    return cv.cvtColor(imgr, cv.COLOR_BGR2RGB), mask_f1, mask_f2, mask_f3

In [ ]:
def revision(img, mask):
	f= 0; i= 0; posh_0=0; posh_1= 0
	posv_1= 0; posv_0= 0
	while i < mask.shape[1]:
		if np.sum(mask[:, i])> 0 and f== 0:
			posh_0= i
			f= 1
		
		if np.sum(mask[:, i])== 0 and f== 1 or i== (mask.shape[1]-1):
			posh_1= i
			break
		i += 1

	f= 0; i= 0
	while i < mask.shape[0]:
		if np.sum(mask[i, :])> 0 and f== 0:
			posv_0= i
			f= 1
		
		if np.sum(mask[i, :])== 0 and f== 1 or i== (mask.shape[0]-1):
			posv_1= i
			break
		i += 1

	return [posv_0, posv_1, posh_0, posh_1]

In [ ]:
def super_pixel(img, n_s, sig, mostrar= False):
    segmentos= slic(img_as_float(img), n_segments= n_s, sigma= sig, start_label= 1)
    
    if mostrar== True:
        fig= plt.figure("Super pixeles")
        ax= fig.add_subplot(1, 1, 1)
        ax.imshow(mark_boundaries(img_as_float(cv.cvtColor(img, cv.COLOR_BGR2RGB)), segmentos))
        plt.axis("off")
        plt.show()
    
    return segmentos